## Import Packages

In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
database_file_location = "../../static/data/transportation.db"

## Collect data into dataframe of format:
* columns = ["rank", "city", "june2020", "2021", etc]
* index = "rank"

In [2]:
df_populations = pd.read_json('../../static/data/city_locations_records.json')


In [3]:
df_populations.head()

,Rank,Geographic Area,City,State,June2020,2020,2021,2022,long,lat
0,1,"New York city, New York",New York,New York,8804194,8740647,8459001,8335897,-74.006015,40.712728
1,2,"Los Angeles city, California",Los Angeles,California,3898767,3895836,3837986,3822238,-118.242766,34.053691
2,3,"Chicago city, Illinois",Chicago,Illinois,2746352,2740076,2698029,2665039,-87.624421,41.875562
3,4,"Houston city, Texas",Houston,Texas,2301572,2299867,2291655,2302878,-95.367697,29.758938
4,5,"Phoenix city, Arizona",Phoenix,Arizona,1608190,1612337,1625356,1644409,-112.074141,33.448437


In [4]:
df_populations.rename(columns={"City":"city", "State":"state", "June2020":"june2020", "Rank":"rank", '2020': "y2020", "2021":"y2021", "2022":"y2022", "long":"lon", "Geographic Area":"region"}, inplace=True)


In [5]:
df_populations.columns

Index(['rank', 'region', 'city', 'state', 'june2020', 'y2020', 'y2021',
       'y2022', 'lon', 'lat'],
      dtype='object')

In [6]:
df_populations.set_index("rank", inplace=True)

In [7]:
df_populations.head()

,region,city,state,june2020,y2020,y2021,y2022,lon,lat
rank,,,,,,,,,
1,"New York city, New York",New York,New York,8804194,8740647,8459001,8335897,-74.006015,40.712728
2,"Los Angeles city, California",Los Angeles,California,3898767,3895836,3837986,3822238,-118.242766,34.053691
3,"Chicago city, Illinois",Chicago,Illinois,2746352,2740076,2698029,2665039,-87.624421,41.875562
4,"Houston city, Texas",Houston,Texas,2301572,2299867,2291655,2302878,-95.367697,29.758938
5,"Phoenix city, Arizona",Phoenix,Arizona,1608190,1612337,1625356,1644409,-112.074141,33.448437


In [8]:
# check for unique index
df_populations.iloc[df_populations.index.duplicated(keep=False), :]

,region,city,state,june2020,y2020,y2021,y2022,lon,lat
rank,,,,,,,,,


## Add records to database

In [9]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [10]:
def create_record(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO population('rank', 'region', 'city', 'state', 'june2020', 'y2020','y2021', 'y2022', 'lon', 'lat')
              VALUES(?,?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

In [11]:
def main():
    database = database_file_location
    
    # create a database connection
    conn = create_connection(database)
    
    for row in df_populations.itertuples():
        print(row)
        with conn:
            # create a new record
            record = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9]);
            record_id = create_record(conn, record)


In [12]:
if __name__ == '__main__':
    main()

Pandas(Index=1, region='New York city, New York', city='New York', state='New York', june2020=8804194, y2020=8740647, y2021=8459001, y2022=8335897, lon=-74.0060152, lat=40.7127281)
Pandas(Index=2, region='Los Angeles city, California', city='Los Angeles', state='California', june2020=3898767, y2020=3895836, y2021=3837986, y2022=3822238, lon=-118.242766, lat=34.0536909)
Pandas(Index=3, region='Chicago city, Illinois', city='Chicago', state='Illinois', june2020=2746352, y2020=2740076, y2021=2698029, y2022=2665039, lon=-87.6244212, lat=41.8755616)
Pandas(Index=4, region='Houston city, Texas', city='Houston', state='Texas', june2020=2301572, y2020=2299867, y2021=2291655, y2022=2302878, lon=-95.3676974, lat=29.7589382)
Pandas(Index=5, region='Phoenix city, Arizona', city='Phoenix', state='Arizona', june2020=1608190, y2020=1612337, y2021=1625356, y2022=1644409, lon=-112.074141, lat=33.4484367)
Pandas(Index=6, region='Philadelphia city, Pennsylvania', city='Philadelphia', state='Pennsylvania'